In [1]:
import pandas as pd
import matplotlib as plot
import numpy as np
import sys
import os

In [2]:
#import data from csv
base_path = "/Users/apple/Documents/gatech/humana_analytics/humana_2018/" # path to humana folder

data = pd.read_csv(base_path + "/data/TAMU_FINAL_DATASET_2018.csv")

In [3]:
data.head()
data.describe() 
#contains one lakh rows

,ID,AGE,AMI_FLAG,ORIG_REAS_ENTITLE_CD,RECON_MA_RISK_SCORE_NBR,RECON_RX_RISK_SCORE_NBR,CON_VISIT_04_Q01,CON_VISIT_04_Q02,CON_VISIT_04_Q03,CON_VISIT_04_Q04,...,COL,COL_GAP,AMM,AMM_GAP,DIAB_PASS,ACE_PASS,STATIN_PASS,ACE_ELIG,DIAB_ELIG,STATIN_ELIG
count,100000.000000,100000.000000,100000.000000,99970.000000,100000.000000,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,...,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.00000,100000.00000,100000.000000
mean,50000.500000,72.770440,0.027260,0.289367,1.138959,1.028531,0.44411,0.446700,0.448490,0.446560,...,0.520610,0.160920,0.027480,0.012910,0.131820,0.389160,0.369960,0.46998,0.16429,0.464400
std,28867.657797,9.715514,0.162841,0.455188,0.946879,0.758254,1.34770,1.315118,1.392324,1.379421,...,0.499578,0.367459,0.163478,0.112887,0.338297,0.487562,0.482796,0.49910,0.37054,0.498734
min,1.000000,40.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000
25%,25000.750000,68.000000,0.000000,0.000000,0.522000,0.570000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000
50%,50000.500000,73.000000,0.000000,0.000000,0.862000,0.891000,0.00000,0.000000,0.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000
75%,75000.250000,79.000000,0.000000,1.000000,1.414000,1.306000,0.00000,0.000000,0.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.00000,0.00000,1.000000
max,100000.000000,95.000000,1.000000,3.000000,15.120000,15.704000,88.00000,70.000000,52.000000,67.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.00000,1.000000


Take Aways -

1. One Lakh rows
2. Check for NAs, because there are - Impute them!
3. Age distribution of the population is on the older side - min age is 40, max is 95

In [4]:
#Cleaning the dataset a bit to make it more general

#standardise column names
data.columns = [s.lower() for s in data.columns]

# find columns with na values
def showmissing(df_train):
    return df_train.columns[df_train.isnull().any()].tolist()

#missing data percentages
(data[showmissing(data)].isnull().sum().sort_values(ascending = False)/100000)*100


diab_type                        64.693
decile_struggle_med_lang         25.215
population_density_centile_us    12.179
online_purchaser                 12.179
dwelling_type                    12.179
education_level                  12.179
length_residence                 12.179
population_density_centile_st    12.179
num_person_household             12.179
college                          12.179
est_bmi_decile                   12.179
online_user                      12.179
pct_above_poverty_line           12.179
pct_below_poverty_line           12.179
home_value                       12.179
est_net_worth                    12.179
est_income                       12.179
index_health_ins_engage          12.179
index_health_ins_influence       12.179
pcp_assignment                    0.157
mco_hlvl_plan_cd                  0.033
mco_prod_type_cd                  0.033
hospice_ind                       0.032
esrd_ind                          0.032
lis                               0.030
